<a href="https://colab.research.google.com/github/Ank1taS/Brain_Tumor_Detection/blob/main/brain_tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>